In [1]:
import torch

In [2]:
from model import GptLanguageModel
from common import GptConfig

/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
hyperparameters = GptConfig()

In [4]:
model = GptLanguageModel(hyperparameters)
model.load_state_dict(torch.load('model_weights.pth', map_location=torch.device('cpu')))
m = model.to(model.device)
tokenizer = m.tokenizer

In [5]:
start_str = """Once upon a time there was a pumpkin. It was a very special pumpkin, it could speak. 
It was sad because it couldn't move. Every day, it would say"""
encoded_input = tokenizer.encode(start_str)
idx = torch.tensor(encoded_input, dtype=torch.long, device=model.device).unsqueeze(0)

In [6]:
print(tokenizer.decode(m.generate(idx = idx, max_new_tokens=model.block_size-len(encoded_input))[0].tolist()))

Once upon a time there was a pumpkin. It was a very special pumpkin, it could speak. 
It was sad because it couldn't move. Every day, it would say, �™mower! We should take turns. It would go very long to the garden and it could not be very well for everyone to do it. Everyone in town was happy, the pumpkin had so much joy to learn more and learn how. It kept its special day. Everyone had fun with its new glove. The red pumpkin was always happy, because the pumpkin knew what would make her very proud of. The moral value that the story has a special story, and that everyone will love the pumpkin forever. And it was never ever alone again - the end to the world and the special pumpkin.
Everyone had so glad to be able on life to be happy, because the old flower had always been so kind and generous! And the little village happy ending to have made it happy, and everyone loved it.
It had learned how important the pumpkin was. The End of that day. And from it was always happy.  
The little bo

In [7]:
from common import TinyStoriesLoader, compute_perplexity
validation_set = TinyStoriesLoader(hyperparameters, split='validation')
print(compute_perplexity(m, validation_set))


Repo card metadata block was not found. Setting CardData to empty.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('roneneldan/TinyStories-33M')
validation_set = TinyStoriesLoader(hyperparameters, split='validation')

In [ ]:
compute_perplexity(model, validation_set)

In [ ]:
start_str = """Once upon a time there was a boy named Gin. He was a very big guy"""
encoded_input = tokenizer.encode(start_str)
idx = torch.tensor(encoded_input, dtype=torch.long, device=model.device).unsqueeze(0)

In [ ]:
print(tokenizer.decode(m.generate(idx = idx, max_new_tokens=model.block_size-len(encoded_input))[0].tolist()))

Once upon a time there was a boy named Gin. He was a very big guy and was playing near the sky! Suddenly the clouds got darker and darker, there could hear a loud noise coming out from outside. The wind made it go away, and everyone started the wind blowing through it.
But the wind came in a bad way and everyone could not. They all started playing. Everyone was scared and the boy didn's work away. The rain got louder than ever and faster. The wind made his friends smile as she moved around in their arms as fast until they could play with the storm! They were very scared of their adventure and started the wind to get away too.
They ran until their rain was all away and they ran around, feeling safe in the rain again. After the storm was time, everyone had a lot to go to the hospital to stay safe, because they knew they were ready for their visit to go back. The storm was a special journey, so they had to stay in the storm. The little person had to stay with them for themselves, no matte